In [ ]:
import requests
import pandas as pd
import geopandas as gpd
import folium
import os
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, shape


key1 = input('Номер трактора')
key2 = input('Дата початку у форматі 2022-03-26: ')
key3 = input('Дата закінчення у форматі 2022-03-26: ')

outName = "%s.shp" % key1
outNameLine = "Line_%s.shp" % key1
outpath = "%s" % key1

print("Намагаюсь намалювати трек трактора %s ....." % key1)

url = "https:_________________________number=%s&dateStart=%sT00:45:45&dateFinished=%sT20:00:00" % (key1, key2, key3)
r = requests.get(url)
json = r.json()
df = pd.DataFrame(json['points'])
gdf = gpd.GeoDataFrame(json['points'])

from shapely.wkt import loads
gdf.geometry =  gdf['point'].apply(loads)
gdf.crs = {'init': 'epsg:4326'}


path = key1
outFolder = r".\Tracks\трек"
resultFolder = os.path.join(outFolder, path)
if not os.path.exists(resultFolder):
    os.makedirs(resultFolder)
output_fp = resultFolder
gdf.to_file(output_fp)

gdf=gdf.rename(index=str, columns={"date":"Date"})
columns = ['Date', 'geometry', 'Name', 'Date_trans']
gdf = gdf.reindex(columns=columns)
gdf['Name'] = key1
gdf['Date_trans'] = gdf['Date'].replace("Т", " ")

outpathLine = os.path.join(resultFolder, outNameLine)

geo_df2 = gdf.groupby(['Name'])['geometry'].apply(lambda x: LineString(x.tolist()))
geo_df2 = gpd.GeoDataFrame(geo_df2, geometry='geometry')
geo_df2.crs = {'init': 'epsg:4326'}
geo_df2.to_file(outpathLine, driver='ESRI Shapefile', encoding = 'utf-8')


m = folium.Map([49.213306, 31.145625], zoom_start=6)

folium.Choropleth(
    geo_df2[geo_df2.geometry.length>0.001],
    line_weight=1,
    line_color='blue'
).add_to(m)

m